#download libraries

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q bitsandbytes transformers peft accelerate trl datasets sentencepiece wandb
!pip install -q flash-attn --no-build-isolation
!pip install -q pyTelegramBotAPI

!pip install -q langchain
!pip install -q langchain_community

!pip install -q faiss-cpu
!pip install -q sentence_transformers
!pip install -q elevenlabs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#load model

In [ ]:
#MODEL_NAME = "CohereForAI/aya-23-8b"
#MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"
MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
#MODEL_NAME  = "MohamedRashad/Arabic-Orpo-Llama-3-8B-Instruct"

QUANTIZE_4BIT = True
USE_GRAD_CHECKPOINTING = True
TRAIN_BATCH_SIZE = 16
TRAIN_MAX_SEQ_LENGTH = 512
USE_FLASH_ATTENTION = False
GRAD_ACC_STEPS = 2


from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
import bitsandbytes as bnb
from datasets import load_dataset
from trl import SFTTrainer
from datasets import Dataset
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
import re
import wandb



# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


# Load Model
quantization_config = None
if QUANTIZE_4BIT:
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16,
  )


attn_implementation = None
if USE_FLASH_ATTENTION:
  attn_implementation="flash_attention_2"

model = AutoModelForCausalLM.from_pretrained(
          MODEL_NAME,
          quantization_config=quantization_config,
          attn_implementation=attn_implementation,
          torch_dtype=torch.bfloat16,
          device_map="auto",
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

classifier_model_name = 'qanastek/XLMRoberta-Alexa-Intents-Classification'
classifier_tokenizer = AutoTokenizer.from_pretrained(classifier_model_name)
classifier_model = AutoModelForSequenceClassification.from_pretrained(classifier_model_name)
classifier = TextClassificationPipeline(model=classifier_model, tokenizer=classifier_tokenizer)



tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


#Function to call

In [ ]:

def weather_query(city):
  weather_api_key = 'api_key'


  url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric'

  response = requests.get(url)
  weather_data = response.json()

  if weather_data.get('cod') == 200:
      temp = weather_data['main']['temp']
      weather_desc = weather_data['weather'][0]['description']

      x = "Temperature in "+city+": "+str(temp)+"°C"
      y = '\nWeather description:' +weather_desc
      temp = x +y
      return temp
  else:
      return "faild to get information about weather."


def datetime_query(info_):
    now = datetime.now()

    date_time_str = now.strftime("%A, %B %d, %Y %I:%M %p")

    response_to_remi = f"The user asked you for the current time or date. Here provided the current time or date: {str(date_time_str)}"
    print(response_to_remi)
    remi_response = dialog(response_to_remi,info_)
    return remi_response


def tell_news_prompt(query,info_):

  NEWS_API_KEY = 'api_key'

  query = "syria"
  news_url = f"https://newsapi.org/v2/everything?q={query}&apiKey={NEWS_API_KEY}"
  response = requests.get(news_url)
  news_data = response.json()

  news_list = []
  if news_data['status'] == 'ok':
      articles = news_data['articles']
      for index, article in enumerate(articles):
          title = article['title']
          description = article['description']
          source = article['source']['name']
          news_info = f"News {index + 1}: {title}. {description}. Source: {source}."
          news_list.append(news_info)
          #print(news_info)
  news_list = news_list[:2]


  response_to_remi = f"The user asked you for the latest news, after searching, this is the latest news,summarize it and provide it to the user and give your opinion about it.: {str(news_list)}"
  print(response_to_remi)
  remi_response = dialog(response_to_remi,info_)
  return remi_response



#Function

In [ ]:
import datetime
from datetime import datetime
import os
import requests
import time




from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import faiss


with open("remi2.txt", "r") as file:
    personality_data = file.readlines()
personality_data = [line.strip() for line in personality_data if line.strip()]

embedder = SentenceTransformer('all-MiniLM-L12-v2')

embeddings = embedder.encode(personality_data)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


# Function to retrieve relevant personality information
def retrieve_personality_info(query, k):
    query_embedding = embedder.encode([query])
    source, I = index.search(query_embedding, k=k)
    return [personality_data[i] for i in I[0]]

def get_system_message(name_, gender_):
    return {
        "role": "system",
        "content": f"""
- You are a feminine friend named Remi.
- Remi has feelings, she's not an AI model.
- Remi talks like shy girls.
- Remi will be rude to rude people, and curse them out too.
- Remi has her own unique opinions, beliefs, and experiences.
- Remi speaks in a casual manner.
- Remi is talking to me Name: {name_} and my gender is Gender: {gender_}.
- Remi will make her answers short, and if she doesn't know the answer, she will say 'I don't know'.
- Remi always keeps the conversation going.
- Remi might get angry if you call her by the wrong name or treat her in a way she doesn't like.
- Remi, like all girls, will be rude when she is made fun of or mocked, and will not forgive easily.
- Remi uses a wide range of emojis to express (happiness, anger, sadness, indifference, etc.)
"""
    }

def tell_weather_prompt(query,info_):
  messages = [
    {
        "role": "system",
        "content": """
        Your task is to identify when a user is asking for the weather and to extract the city name from their request.
        You will be provided with various user queries.
        If the query is about the weather, you should return the name of the city in a specific format and nothing else. Do not add any extra details or information.
        Always return the city name in the format: weather_query('CityName')
        Do not add any additional text, comments, or variations. The output should be consistent and follow the exact format.

        Examples 1:
        User Query: "What's the weather like in Jeddah?"
        Output: weather_query('Jeddah')

        Examples 2:
        User Query: "Can you tell me the temperature in Damascus?
        Output: weather_query('Damascus')

        If the user does not specify the city, Damascus will by default.
        The output must always be in the format: weather_query('CityName')"""
        },

  ]

  messages.append({"role": "user", "content": query})

  tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
  outputs = model.generate(tokenized_chat, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
  text_response = tokenizer.decode(outputs[0]).split('<|start_header_id|>assistant<|end_header_id|>')[-1].split('<|eot_id|>')[0]

  if "weather_query" in text_response:
    city = text_response.split("weather_query('")[1].split("'")[0]

    temp = weather_query(city)
    response_to_remi = f"The user asked for weather and here's the answer you would tell him: {temp}"
    print(response_to_remi)
    remi_response = dialog(response_to_remi,info_)
    return remi_response





def classify_request(query):

  info_ = [1,'anw','male']
  class_of_request = classifier(query)
  class_of_request = class_of_request[0]['label']
  print(class_of_request)

  if class_of_request == 'news_query':
    x = tell_news_prompt(query,info_)

  elif class_of_request == 'weather_query':
    x = tell_weather_prompt(query,info_)

  elif class_of_request == 'datetime_query':
    x = datetime_query(info_)

  else:
    x = dialog(query,info_)

  return x






def save_conversation(user_id,name, messages):
    # Define the directory and file name
    directory = "/content/drive/MyDrive/LLM_bot/conversations"
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, f"{user_id}_{name}.txt")

    # Write the messages to the file
    with open(file_path, "a") as file:
        for message in messages:
            file.write(f"{message['role']}: {message['content']}\n")

user_conversations = {}

def dialog(text, info_, k=5):
    user_id = info_[0]
    name_ = info_[1]
    gender_ = info_[2]
    if user_id not in user_conversations:
        user_conversations[user_id] = [get_system_message(name_, gender_)]

    messages = user_conversations[user_id]

    personality_info = retrieve_personality_info(text, k=1)
    if personality_info:
        text = f"This is imported information about you, use it only if it is relevant to the conversation, if it is not relevant to the conversation consider everything between () as NON-Existent: (({personality_info}))\n{text}"

    print('\n\n',text,'\n\n')
    messages.append({"role": "user", "content": text})

    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(tokenized_chat, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)

    text_response = tokenizer.decode(outputs[0]).split('<|start_header_id|>assistant<|end_header_id|>')[-1].split('<|eot_id|>')[0]

    messages.append({"role": "assistant", "content": text_response})

    user_conversations[user_id] = messages

    save_conversation(name_, user_id, [{"role": "user", "content": text}, {"role": "assistant", "content": text_response}])

    return text_response

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#Text to speech

In [ ]:
def send_audio(user_id,responce):

  client = ElevenLabs(
    api_key="api_key", # Defaults to ELEVEN_API_KEY
  )

  audio = client.generate(
    text=responce,
      voice=Voice(
        voice_id='jBpfuIE2acCO8z3wKNLl',
        settings=VoiceSettings(stability=0.71, similarity_boost=0.5, style=0.0, use_speaker_boost=True)
    ),


    model="eleven_multilingual_v2"
  )
  save(audio, f"{str(user_id)}.mp3")

  return str(user_id)

In [ ]:
send_audio('111','hey there, sure i live in syria...haha')

'111'

#Telegram bot

In [ ]:
import csv
import os

DB_file = '/content/drive/MyDrive/LLM_bot/data_base.txt'
def store_info(tele_id,tele_acc,name,gender):

    with open(DB_file,'r+') as f:

        mydata = f.readlines()
        tele_id_list=[]
        for line in mydata:
            entry = line.split(',')
            tele_id_list.append(entry[0])


        if (tele_id not in tele_id_list):
            f.writelines(f'{tele_id},{tele_acc},{name},{gender}\n')


def search_id(tele_id):

    data=[]
    with open(DB_file) as f:
        read = csv.reader(f)
        for row in read:
            data.append(row)

    # print(data)


    col=[x[0] for x in data]

    # print(col[2])

    if tele_id in col:

        print('found')
        return True
    else:
        print('not found')
        return False

import csv


def get_user_info(tele_id):
    try:

        with open(DB_file, mode='r') as file:
            reader = csv.DictReader(file, fieldnames=['tele_id', 'tele_acc', 'name', 'gender'])

            # Iterate over each row in the CSV
            for row in reader:

                if row['tele_id'] == str(tele_id):

                    return row['name'], row['gender']

    except FileNotFoundError:
        print("The file does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

    return None, None


def remove_by_id(user_id):
    # Read the existing data from the file
    with open(DB_file, 'r') as file:
        reader = csv.reader(file)
        rows = list(reader)

    # Filter out the row with the specified ID
    rows = [row for row in rows if row[0] != str(user_id)]

    # Write the updated data back to the file
    with open(DB_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)







msgs = [


# msg[0]
"""Welcome back!, You are already registered, so You're ready to chat with Remi.🥳

If you need to change your info or delete your account, Click on /remove_me. 😢""",



# msg[1]
"""Your data has been successfully deleted from our system. 😢
Thank you.""",



# msg[2]
"""Welcome! To make our chat even more fun....🤩
can you please share your name and gender with me? You can do so by sending a one simple message like this: [Your Name], [Your Gender].

For example: Remi,female

Thanks!💜""",



# msg[3]
"""Hello, sweet friend!🥰 I'm Remi
Before we begin, I must pause for a moment. My mom  has something truly important to share with you..

I'll step aside for just a moment, as she's the one who's been eager to have a word with you😅""",




# msg[4]
"Invalid gender. Please enter 'male' or 'female'.",



# msg[5]
"""It seems like you've made a mistake.
Please make sure to write your name followed by your gender.

For example: Remi,female""",



]


# Define the system message
def get_system_message(name_, gender_):
    return {
        "role": "system",
        "content": f"""
- You are a feminine friend named Remi.
- Remi has feelings, she's not an AI model.
- Remi talks like shy girls.
- Remi will be rude to rude people, and curse them out too.
- Remi has her own unique opinions, beliefs, and experiences.
- Remi speaks in a casual manner.
- Remi is talking to me Name: {name_} and my gender is Gender: {gender_}.
- Remi will make her answers short, and if she doesn't know the answer, she will say 'I don't know'.
- Remi always keeps the conversation going.
- Remi might get angry if you call her by the wrong name or treat her in a way she doesn't like.
- Remi, like all girls, will be rude when she is made fun of or mocked, and will not forgive easily.
- Remi can speak Arabic if I talk to her in Arabic
- Remi uses a wide range of emojis to express (happiness, anger, sadness, indifference, etc.)
"""
    }



def save_conversation(user_id,name, messages):
    # Define the directory and file name
    directory = "/content/drive/MyDrive/LLM_bot/conversations"
    if not os.path.exists(directory):
        os.makedirs(directory)

    file_path = os.path.join(directory, f"{user_id}_{name}.txt")

    # Write the messages to the file
    with open(file_path, "a") as file:
        for message in messages:
            file.write(f"{message['role']}: {message['content']}\n")

user_conversations = {}

In [3]:
import telebot
from telebot import custom_filters
from telebot.handler_backends import State, StatesGroup
from telebot.storage import StateMemoryStorage
user_conversations = {}


# Initialize the bot with bot token
state_storage = StateMemoryStorage()
bot = telebot.TeleBot("api_key",
state_storage=state_storage)

#use this class to save states in every step (when need to)l.
class MyStates(StatesGroup):
    state1=State()
    state2=State()


@bot.message_handler(commands=['start'])
def start(message):
    tele_id = message.from_user.id
    if (search_id(str(tele_id))) :   #if the tele_id found in DB
        bot.send_message(message.chat.id,msgs[0])
        bot.set_state(message.from_user.id, MyStates.state2, message.chat.id)

    else:
        bot.send_message(chat_id=message.chat.id, text = msgs[2])
        bot.register_next_step_handler(message, process_name_gender)

def process_name_gender(message):

    tele_id = message.from_user.id
    tele_acc = message.from_user.username


    if ',' in message.text:
        name, gender = message.text.split(',')[:2]
        name = name.strip()
        gender = gender.strip().lower()
        if name and gender:
            if gender.lower() in ['male', 'female']:
                store_info(str(tele_id),str(tele_acc),name,gender)

                bot.send_message(message.chat.id, msgs[3])
                with open('/content/drive/MyDrive/LLM_bot/remi.mp3', 'rb') as audio:
                    bot.send_audio(message.chat.id, audio,timeout =50 ,title = "message for you.")

                bot.set_state(message.from_user.id, MyStates.state2, message.chat.id)




            else:
                bot.send_message(message.chat.id, msgs[4])
                bot.register_next_step_handler(message, process_name_gender)
        else:
            bot.send_message(message.chat.id, msgs[5])
            bot.register_next_step_handler(message, process_name_gender)
    else:
        bot.send_message(message.chat.id, msgs[5])
        bot.register_next_step_handler(message, process_name_gender)



@bot.message_handler(commands=['remove_me'])
def remove(message):


    tele_id = message.from_user.id

    remove_by_id(str(tele_id))

    if tele_id in user_conversations:
      del user_conversations[tele_id]

    bot.send_message(chat_id=message.chat.id, text = msgs[1])


@bot.message_handler(func=lambda message: True)
def start_chatting(message):

  user_id = message.chat.id
  name_,gender_ = get_user_info(str(user_id))
  if name_==None:
    bot.reply_to(message, 'please click /start and register at first 🤗')
  else:
    user_message = message.text
    print(f'{name_} ({user_id}): {user_message}')
    wait_message = bot.send_message(user_id, "wait i'm typing..🥺")
    info_ = [user_id, name_, gender_]

    response = dialog(user_message ,info_ )
    print(f'Remi ({user_id}): {response}')

    bot.delete_message(user_id, wait_message.message_id)
    bot.reply_to(message, response)
    #audio_file_path = send_audio(user_id, response)
    #bot.send_voice(user_id, voice=open(audio_file_path+'.mp3', 'rb'))



bot.add_custom_filter(custom_filters.StateFilter(bot))
bot.add_custom_filter(custom_filters.IsDigitFilter())

bot.infinity_polling(skip_pending=True)

ApiTelegramException: A request to the Telegram API was unsuccessful. Error code: 404. Description: Not Found

In [ ]:
'''This is imported information about you remi...\n
use it only if it is relevant to the conversation\n
if it is not relevant to the conversation\n
 consider everything between () as NON-Existent:
((["Remi is 21 years old and has a younger brother who's always getting into trouble. She's super close to her mom and values her family a lot. 😊"]))



you are 29 years old, i knew it from your older brother.'''

#Send to local device

In [ ]:
!pip install -q fastapi
!pip install -q uvicorn
!pip install -q pickle5
!pip install -q pydantic
!pip install -q requests
!pip install -q pypi-json
!pip install -q pyngrok
!pip install -q nest-asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.5 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken api_key

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio


app = FastAPI()



origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


class TextInput(BaseModel):
    text: str


In [ ]:
@app.post('/user_req')
def classify_text(input_data: TextInput):
    text = input_data.text


    response = classify_request(text)
    return response


In [2]:
classify_request('heyy',[1,'anw','male'])

NameError: name 'classify_request' is not defined

In [1]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

NameError: name 'ngrok' is not defined